In [1]:
!pip install python-dotenv pdfplumber

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 42.8/42.8 kB 1.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 48.2/48.2 kB 3.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 60.2/60.2 kB 932.3 kB/s eta 0:00:00 0:00:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.6/5.6 MB 21.4 MB/s eta 0:00:0000:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 4.5/4.5 MB 23.9 MB/s eta 0:00:0000:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.8/2.8 MB 16.4 MB/s eta 0:00:0000:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 4.0/4.0 MB 36.0 MB/s eta 0:00:0000:0100:01


In [2]:
import os
import requests
import pdfplumber
import re
import json

In [5]:
url = "https://beonbrand.getbynder.com/m/17db51e853b2fca1/"

In [9]:
# Extraction functions
def extract_title(lines):
    '''Extracts title from the second line of the input, the input is pdf text split by \n'''
    title = ""
    if len(lines) > 1:
        try:
            title = lines[1]
        except ValueError:
            print("Second line is not a number.")
    return title

def extract_rules1(lines):
    '''Extracts rules from the fraction between MEDICAL POLOICY CRITERIA and POLICY GUIDELINES of input, the input is pdf tect split by \n'''
    # search between signs
    start_sign = "MEDICAL POLICY CRITERIA"
    end_sign = "NOTE: A summary of the supporting rationale for the policy criteria is at the end of the policy."
    start_index = lines.index(start_sign)
    end_index = lines.index(end_sign)

    if start_index == -1 and end_index == -1:
        return {}

    #

    ed = lines[start_index+1:end_index]
    rules = {"rule_id": "1", "rule_text": "Coverage Criteria", "operator": "AND", "rules": []}
    level_dic = {1: 0, 2: 0, 3: 0} # Use of dic to save memory
    last_matched = None

    for line in ed:
        # Match lines starting with Roman numerals or numbers (e.g., I., II., A., 1.)
        matches = re.match(r'^([IVX]+|[A-Z]+|\d+)\.\s+(.*)', line)

        if matches:
            level = matches.group(1)  # Extract level (Roman, number, or letter)
            content = matches.group(2)  # Extract content
            l_id = 0
            rule_id = ""
            operator = None
            current_level = rules["rules"]

            # IDs
            if re.match(r'^[IVX]', level):
                l_id = 1
                opeartor = "OR"
                l1_rule_id = level_dic.get(1) + 1
                rule_id = "1." + str(l1_rule_id)
                level_dic.update({1: l1_rule_id, 2: 0, 3: 0})
            elif re.match(r'^[A-Z]', level):
                l_id = 2
                operator = "OR"
                l2_rule_id = level_dic.get(2) + 1
                rule_id = "1." + str(level_dic.get(1)) + "." + str(l2_rule_id)
                level_dic.update({2: l2_rule_id,3: 0})
            elif re.match(r'^\d', level):
                l_id = 3
                l3_rule_id = level_dic.get(3) + 1
                rule_id = "1." + str(level_dic.get(1)) + "." + str(level_dic.get(2)) + "." + str(l3_rule_id)
                level_dic.update({3: l3_rule_id})
            else:
                print("Rule Extract Error: Cannot identify the sign")

            content = re.sub(r'\s+and$|\s+or$', '', content)
            obj = {"rule_id": rule_id, "rule_text": content, "operator": opeartor, "rules": None}
            last_matched = obj

            if l_id == 2:
                if rules["rules"][level_dic.get(1)-1]["rules"]:
                    current_level = rules["rules"][level_dic.get(1)-1]["rules"]
                else:
                    current_level = []
                    rules["rules"][level_dic.get(1)-1]["rules"] = [current_level]
            elif l_id == 3:
                print(rules["rules"][level_dic.get(1)-1])
                if rules["rules"][level_dic.get(1)-1]["rules"][level_dic.get(2)-1]["rules"]:
                    current_level = rules["rules"][level_dic.get(1)-1]["rules"][level_dic.get(2)-1]["rules"]
                else:
                    current_level = []
                    rules["rules"][level_dic.get(1)-1]["rules"][level_dic.get(2)-1]["rules"] = current_level
            current_level.append(obj)
        else:
            if last_matched:
                line = re.sub(r'\s+and$|\s+or$', '', line)
                last_matched["rule_text"] += " " + line

    return rules

def extract_codes2(lines):
    '''Extracts code from the fraction between the first existed Code Number Description and Date of Origin of input, the input is pdf tect split by \n'''
    # search between signs
    cost = "Codes Number Description"
    end_sign = "Date of Origin"
    start_index = lines.index(cost)
    end_index = next((i for i, line in enumerate(lines) if line.startswith(end_sign)), None)

    if start_index == -1 and end_index == -1:
        return {}

    ed = lines[start_index+1:end_index]
    codes = {"cpt": None, "icd10": None, "hcpcs": None}
    current_group = None
    current_code_number_a = None

    for line in ed:
        # Match lines starting with Roman numerals or numbers (e.g., I., II., A., 1.)
        matches_1 = re.match(r'^(CPT|ICD10|HCPCS)+\s+([\dA-Z]{5})+\s+(.*)', line)

        if matches_1:
            group = matches_1.group(1)
            number = matches_1.group(2)
            des = matches_1.group(3)

            # As
            if re.match(r'^CPT', group):
                current_group = "cpt"
            elif re.match(r'^ICD10', group):
                current_group = "icd10"
            elif re.match(r'^HCPCS', group):
                current_group = "hcpcs"
            else:
                print("Rule Extract Error: Cannot identify the sign")

            codes[current_group] = number

    return codes

In [7]:
def extract_pdf(KEY, url):
    # download PDF for extraction
    response = requests.get(url)
    with open("downloaded.pdf", "wb") as f:
        f.write(response.content)
    # open PDF and extract
    text = ""
    with pdfplumber.open("downloaded.pdf") as pdf:
        for page in pdf.pages:
            text_piece = page.extract_text()
            text += text_piece + "\n"
    text = re.sub(r'^SUR.*\n', '', text, flags=re.MULTILINE) # remove page index
    lines = text.split('\n') # line spliting
    cost = "Codes Number Description"
    extracted_title = extract_title(lines)
    extracted_rules = extract_rules1(lines)
    extracted_codes = extract_codes2(lines)
    out = {
        "title": extracted_title,
        "insurance_name": "Regene",
        "rules": extracted_rules,
        "codes": [extracted_codes]}
    return out

In [10]:
txt = extract_pdf(KEY, url)
with open('output.json', 'w') as f:
    json.dump(txt, f, indent=4) # Write the output to a file
    print("please check your $currentFolder/output.json"); f.close()

please check your $currentFolder/output.json


In [ ]:
def extract_codes1(lines):
    '''Extracts code from the fraction between the first existed Code Number Description and Date of Origin of input, the input is pdf tect split by \n'''
    # search between signs
    ''' Different Format'''
    end_sign = "Date of Origin"
    start_index = lines.index(cost)
    end_index = next((i for i, line in enumerate(lines) if line.startswith(end_sign)), None)

    if start_index == -1 and end_index == -1:
        return {}

    ed = lines[start_index+1:end_index]
    print(ed)
    codes = {"cpt": None, "icd10": None, "hcpcs": None}
    current_group = None
    current_code_number_a = None

    for line in ed:
        # Match lines starting with Roman numerals or numbers (e.g., I., II., A., 1.)
        matches_1 = re.match(r'^(CPT|ICD10|HCPCS)+\s+(\d|[A-Z]{5})+\s+(.*)', line)
        matches_2 = re.match(r'^(\d|[A-Z]{5})+\s+(.*)', line)
        matches_3 = re.match(r'^(CPT|ICD10|HCPCS)+\s+None', line)

        if matches_1:
            group = matches_1.group(1)
            number = matches_1.group(2)
            print(number)
            des = matches_1.group(3)

            # As
            if re.match(r'^CPT', group):
                current_group = "cpt"
            elif re.match(r'^ICD10', group):
                current_group = "icd10"
            elif re.match(r'^HCPCS', group):
                current_group = "hcpcs"
            else:
                print("Rule Extract Error: Cannot identify the sign")

            if not codes[current_group]:
                codes[current_group] = []

            obj = {number: des}
            codes[current_group].append(obj)
        elif matches_2:
            if current_group:
                number = matches_2.group(1)
                des = matches_2.group(2)

                codes[current_group].append({number: des})
        elif matches_3:
            continue
        else:
            if current_code_number_a:
                codes[current_group][-1] += line

    return codes
